In [45]:
import numpy as np
import pandas as pd

#### TODO:
- add capacity constraint (may end up being a separate bin packing problem. potentially leverage parallel processing for this and just implement a greedy approach here)
- convert the yummy score to the weighted average preference (weighted by qty of plants of each type)
- add objective to only plant 1-2 unique plants per bed
- add to bed info the list of incompatable plants in that bed
- lookup sun requirements for plants
- weighting of objectives
- weight preferences
- more realistic preferences
- look into parallel processing. potentially have notebook run in AWS
- visualizations at the end
- turn into a class
- look into random seed best practices

In [46]:
plant_info = pd.read_csv('../data/plant_data.csv')
bed_info = pd.read_csv('../data/bed_data.csv')

In [47]:
plant_info.index.name = 'plant_index'
plants = plant_info.plant.to_numpy()
plant_index = plant_info.index.to_numpy()
num_plants = len(plants)

In [48]:
bed_info.index.name = 'bed_index'
beds = bed_info.bed.to_numpy()
bed_index = bed_info.index.to_numpy()
num_beds = len(beds)

bed_width = 3
bed_length = 10

In [49]:
print(num_beds)
print(beds)
print(bed_index)

42
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41]


In [50]:
# #random assignement of sun requirements and sun availability
# plant_info.sun = plant_info.sun.apply(lambda x: np.random.choice(['Full','Partial']))
# bed_info.sun = bed_info.sun.apply(lambda x: np.random.choice(['Full','Partial']))

In [51]:
#average preference
fam = ['evan','gina','liesse','lizzie','jack']
plant_info['avg_pref'] = plant_info[fam].mean(axis=1)
plant_info.drop(fam,axis=1,inplace=True)
preferences = plant_info.avg_pref.to_numpy()
plant_index_by_pref = plant_info.sort_values('avg_pref',ascending=False).index 

plant_spacing = plant_info.plant_spacing.to_numpy()

In [52]:
#get plant and bed sun requirements
plant_sun_req = plant_info.sun.to_numpy()
bed_sun_req = bed_info.sun.to_numpy()

In [ ]:
weights = {'yield_': 0.3333,
           'yummy_score': 0.3333,
           'variety_score': 0.3333}

In [53]:
plant_info.sort_values('avg_pref',ascending=False).iloc[0].name

12

In [54]:
#most preferred 
max_pref = plant_info.sort_values('avg_pref',ascending=False).iloc[0].name

In [55]:
#convert info dfs to dicts
# plant_info = plant_info.to_dict(orient='index')
# bed_info = bed_info.to_dict(orient='index')

In [56]:
#initialize plan. no plants in any bed
np.random.seed(2134)
plan = np.random.randint(0,10,size=(num_beds,num_plants))

#for keeping track of what axis is which
bed_axis = 0
plant_axis = 1

In [57]:
# #generate initial solution using a greedy heuristic. 
# #plant most preferred fruit or veggie and plant everywhere

# def initial_solution(plan,max_pref):

#     initial_solution = plan.copy()
#     return initial_solution

In [58]:
def make_neighbor(plan):
    #pick a random bed
    bed = np.random.choice(bed_index)
    #pick a random plant
    plant = np.random.choice(plant_index)
    #decide whether to plant another plant in bed or remove one
    decision = np.random.choice([-1,1],p=[0.2,0.8])

    #make a decision
    new_plan = plan.copy()
    new_plan[bed][plant] = max(0,new_plan[bed][plant] + decision) #max(0,n) takes care of case of negative planting

    return new_plan

In [59]:
def sun_constraint(plan,bed_sun_req,plant_sun_req):
    """
    Scans beds and plants, kills any plant that is in a bed that does not match its sun requirement.
    """
    plan_sun = plan.copy()
    for b in bed_index:
        for p in plant_index:
            b_sun = bed_sun_req[b]
            p_sun = plant_sun_req[p]
            if b_sun != p_sun:
                plan_sun[b][p] = 0
    return plan_sun
plan = sun_constraint(plan,bed_sun_req,plant_sun_req)

In [60]:
bed_length*12/18

6.666666666666667

In [61]:
plan

array([[0, 0, 1, ..., 0, 4, 1],
       [0, 0, 0, ..., 1, 5, 2],
       [0, 0, 1, ..., 6, 0, 0],
       ...,
       [1, 7, 0, ..., 0, 0, 0],
       [0, 6, 0, ..., 0, 0, 0],
       [7, 9, 0, ..., 0, 0, 0]])

In [62]:
def bed_packing(bed,plan,bed_length,plant_index_by_pref,plant_spacing):
    """
    Handles the fact that the plan doesn't account for the spacing constraints required in each bed. Plan is likely 
    over planting inside of each bed. Each bed is now a separate bin packing problem. This function applies a greedy 
    heuristic for updating a plan to prioritize planting most preferred plants until you run out of space. Not
    necessarily optimal because you could improve the objective by planting less preferred plants that require less space.    
    """
    plan_beds_finalized = plan.copy()
    remaining_capacity = bed_length*12

    for plant in plant_index_by_pref:
        if remaining_capacity>0:
            spacing = plant_spacing[plant]
            num_plants_planned = plan[bed][plant]
            num_plant_capacity = remaining_capacity/spacing
            plants_planted = min(num_plants_planned,num_plant_capacity) #ensures I don't exceed capacity
            plan_beds_finalized[bed][plant] = plants_planted
            remaining_capacity -= plants_planted*spacing
        else:
            plan_beds_finalized[bed][plant] = 0 #once I run out of capacity, update plan to plant 0       
    return plan_beds_finalized


In [63]:
plan.sum()

2474

In [66]:
# def yoy_planting_constraint(plan)

In [67]:
def compute_yield(plan):
    """
    Yield is the % of plants that are planted and survive. Not all plants will survive where they are planted.
    This is where we impose constraints on sun requirements, year-over-year planting requirements, and companion planting requirements.
    Returns the total number of surviving plants / the total plants planted.
    """
    # 
    #Too little/too much sun can kill certain plants
    #Some plants don't do well depending on what was planted the year before
    #Some plants don't go well next to each other

    #denominator. total number of plants planted
    total_plants_planted = plan.sum()    

    #impose constraints that kill plants in beds where they are not supposed to be
    plan = sun_constraint(plan,bed_sun_req,plant_sun_req)
    # plan = yoy_planting_constraint(plan)
    # plan = companion_planting_constraint(plan)
    return (plan.sum()/total_plants_planted)*100



In [68]:
def compute_yummy_score(plan,preferences):
    """Multiplies the avg preference for each plant by the quantity of each plant in the plan.
       Maximization encourages plants with higher preferences to be planted in higher quantities."""
    plan_by_plant = plan.sum(axis=bed_axis)
    yums = (plan_by_plant*preferences).sum()
    return yums

In [69]:
def compute_variety_score(plan):
    """Sums the number of unique plants that are actually planted in the garden. Counts the number of plants that are being planted across all beds.
       Then counts the number of plants with non-zero planting plan. 
       Maximization encourages more unique plants to be planted."""
    variety_score = (plan.sum(axis=bed_axis) > 0).sum()
    return variety_score

In [71]:
initial_obj_values

{'yield_': 100.0, 'yummy_score': 13348.599999999999, 'variety_score': 36}

In [1]:
def get_objective(plan,weights,initial_obj_values):
    yield_ = compute_yield(plan)
    yummy_score = compute_yummy_score(plan,preferences)
    variety_score = compute_variety_score(plan)
    objective = (
                (weights['yield_']*yield_) / initial_obj_values['yield_']
              + (weights['yummy_score']*yummy_score) / initial_obj_values['yummy_score'] 
              + (weights['variety_score']*variety_score) / initial_obj_values['variety_score']
                )
    return objective


In [76]:
def optimize(initial_plan,weights,max_iter=10,starting_temperature=10000,alpha=0.99):

    initial_obj_values = {'yield_': compute_yield(plan),
                          'yummy_score': compute_yummy_score(plan,preferences),
                          'variety_score': compute_variety_score(plan)}

    current_plan = initial_plan.copy()
    current_objective = get_objective(initial_plan,weights,initial_obj_values)
    
    best_plan = initial_plan.copy()
    best_obj = current_objective
        
    for i in range(max_iter):
        new_plan = make_neighbor(plan)
        new_obj = get_objective(plan,weights,initial_obj_values)

        #loop through beds and solve sub-problems
        for bed in bed_index: #this loop would be taken out if paralelizing this
            bed_packing(bed,plan,bed_length,plant_index_by_pref,plant_spacing)

        np.random.uniform()

        #lower the temperature
        temperature -= 1
        
    return plan

optimize(plan,weights)

array([[ 0,  0,  0, ...,  0,  4,  1],
       [ 2,  0,  0, ...,  1,  4,  2],
       [ 0,  1,  2, ...,  7,  1,  0],
       ...,
       [ 2,  6,  0, ...,  0,  1,  1],
       [ 1,  6,  2, ...,  1,  0,  1],
       [ 7, 10,  2, ...,  0,  3,  1]])

In [ ]:
#here we can also apply things like sun requirements, yoy planting constraints, same bed planting constraints
    